In [1]:
from ipywidgets import interact

In [4]:
import torch

In [15]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (18,12) # set default size of plots

In [6]:
from training.sdf_experiment import SdfExperiment, Siren
from renderer.camera import Camera
from renderer.renderer import SphereTracingRenderer

In [5]:
state = torch.load('lightning_logs/version_19/checkpoints/epoch=112-step=61924.ckpt')

In [9]:
siren = Siren(3, 512, 5, outermost_linear=True, 
              out_features=1, 
              first_omega_0=50, 
              hidden_omega_0=30, 
              use_geometric_initialization=True)

In [13]:
siren.load_state_dict({k.replace('sdf_model.', ''):v for k,v in state['state_dict'].items()})
siren = siren.cuda()

In [14]:
width, height = 512, 512

In [16]:
@interact
def render(
        dist=(0.5,3, 0.1),
        elev=(-360, 360, 1),
        azim=(-360, 360, 1),
        max_iteration = (1, 150, 1),
        max_depth = (1,5, 1)):
    camera = Camera(dist, elev, azim, resolution=256)
    rend = SphereTracingRenderer(camera, max_iteration=max_iteration,max_depth=max_depth )
    frame = rend.render(siren).detach().cpu().numpy()
    plt.grid(visible=None)
    plt.imshow(frame)
    plt.show() 

interactive(children=(FloatSlider(value=1.7000000000000002, description='dist', max=3.0, min=0.5), IntSlider(v…